In [1]:
""" Replicating the SimpleNet model architecture. """

import keras
import numpy as np
import keras.backend as K
from keras.utils import np_utils
from keras.datasets import cifar10
from keras.models import Sequential
from keras.models import load_model
from keras import regularizers, optimizers
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import glorot_normal, RandomNormal, Zeros
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization

Using TensorFlow backend.


In [2]:
# Data Retrieval & mean/std preprocess
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [3]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
    )
datagen.fit(x_train)

In [4]:
# Define Model architecture
def create_model(s = 2, weight_decay = 1e-2, act="relu"):
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal(), input_shape=x_train.shape[1:]))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 2
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 3
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 4
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    # First Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(Dropout(0.2))
    
    
    # Block 5
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 6
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 7
    model.add(Conv2D(256, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    # Second Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    
    # Block 8
    model.add(Conv2D(256, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 9
    model.add(Conv2D(256, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    # Third Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    
    
    # Block 10
    model.add(Conv2D(512, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))

    # Block 11  
    model.add(Conv2D(2048, (1,1), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 12  
    model.add(Conv2D(256, (1,1), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(Activation(act))
    # Fourth Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(Dropout(0.2))


    # Block 13
    model.add(Conv2D(256, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(Activation(act))
    # Fifth Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))

    # Final Classifier
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    return model

In [5]:
# Prepare for training 
model = create_model(act="relu")
batch_size = 128
epochs = 25
train = {}

# First training for 50 epochs - (0-50)
opt_adm = keras.optimizers.Adadelta(lr=0.1)
model.compile(loss='categorical_crossentropy', optimizer=opt_adm, metrics=['accuracy'])
train["part_1"] = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=2*epochs,
                                    verbose=1,validation_data=(x_test,y_test))
model.save("simplenet_trained_model.h5")
print(train["part_1"].history)

Epoch 1/50
390/390 [==============================] - 87s 222ms/step - loss: 13.3953 - acc: 0.3293 - val_loss: 12.7298 - val_acc: 0.1370  - ETA: 35s - loss: 14.5209 -  - ETA: 33s - loss - E - ETA: 1s - loss: 13.4416
Epoch 2/50
390/390 [==============================] - 81s 208ms/step - loss: 8.8189 - acc: 0.4945 - val_loss: 8.5696 - val_acc: 0.183313 - acc:  - ETA: 5s - loss - ETA: 0s - loss: 8.8369 - acc: 0.494 - ETA: 0s - loss: 8.8326 - acc: 0.4
Epoch 3/50
390/390 [==============================] - 80s 205ms/step - loss: 5.8523 - acc: 0.5925 - val_loss: 5.2709 - val_acc: 0.4896loss: 7.0415 - acc:  - ETA: 1:10 - loss: 7.0098 - acc: 0.548 - ETA: 1:10 - loss: 7.0072 -  - ETA: 1:08 - loss: 6.9498 - acc: - ETA - ETA: 56s  - ETA: 53s - loss: 6.6862 - acc: 0. - ETA: 53s - loss: 6.6793 - acc: 0. - ETA: 53s -  - ETA:  - ETA: 43s - loss: 6.5115 - acc:  - ETA: 42s - loss: 6.50 - ETA: 37s - loss: 6.4072 
Epoch 4/50
390/390 [==============================] - 80s 206ms/step - loss: 4.0654 - acc: 0

In [6]:
# Training for 25 epochs more - (50-75)
opt_adm = keras.optimizers.Adadelta(lr=0.1*0.1)
model.compile(loss='categorical_crossentropy', optimizer=opt_adm, metrics=['accuracy'])
train["part_2"] = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,
                                    verbose=1,validation_data=(x_test,y_test))
model.save("simplenet_trained_model.h5")
print(train["part_2"].history)

Epoch 1/25
390/390 [==============================] - 83s 213ms/step - loss: 0.5630 - acc: 0.9179 - val_loss: 0.5595 - val_acc: 0.92082:17 - los - ETA: 1:34 - loss: 0.603 - ETA: 1:06 - loss: 0.5934 - - ETA: 1:02 - loss - ETA: 58s - loss: 0.5873 - acc:  - ETA: 57s - loss: 0.58 - ETA: 11s - loss: - ETA: 8s - loss: 0.5649 - acc - ETA: 6s - lo - ETA: 1s - loss: 0.5630 - acc: 0. - ETA: 0s - loss: 0.5630 - acc: 0.
Epoch 2/25
390/390 [==============================] - 80s 205ms/step - loss: 0.5339 - acc: 0.9257 - val_loss: 0.5404 - val_acc: 0.9253. - ETA: - ETA:
Epoch 3/25
390/390 [==============================] - 80s 205ms/step - loss: 0.5138 - acc: 0.9314 - val_loss: 0.5480 - val_acc: 0.92178s - loss: 0.5136 - acc: - ETA: 7s - loss: 0.51 - ETA: 4s - los
Epoch 4/25
390/390 [==============================] - 80s 205ms/step - loss: 0.5015 - acc: 0.9334 - val_loss: 0.5185 - val_acc: 0.93042s - loss: 0.501
Epoch 5/25
390/390 [==============================] - 80s 205ms/step - loss: 0.4928 - acc

In [7]:
# Training for 25 epochs more - (75-100)
opt_adm = keras.optimizers.Adadelta(lr=0.1*0.1*0.1)
model.compile(loss='categorical_crossentropy', optimizer=opt_adm, metrics=['accuracy'])
train["part_3"] = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,
                                    verbose=1,validation_data=(x_test,y_test))
model.save("simplenet_trained_model.h5")
print(train["part_3"].history)

Epoch 1/25
390/390 [==============================] - 83s 213ms/step - loss: 0.3784 - acc: 0.9547 - val_loss: 0.4370 - val_acc: 0.9395 0.
Epoch 2/25
390/390 [==============================] - 80s 204ms/step - loss: 0.3737 - acc: 0.9560 - val_loss: 0.4357 - val_acc: 0.9402- ETA: 15s - loss: 0.3749 - ETA: 13s - loss: 0.37 - ETA: 12s -  - ETA: 2s - loss: 0.3739 - acc: - ETA: 0s - loss: 0.3737 - acc: 0.
Epoch 3/25
390/390 [==============================] - 80s 204ms/step - loss: 0.3727 - acc: 0.9564 - val_loss: 0.4369 - val_acc: 0.9399ETA: 1:10 - loss: 0.3772 - acc: 0.96 - ETA: 1:11 - los - ETA: 2s - loss: 0.3728 - acc: 0. - ETA: 1s - loss: 0.3729 - acc: 0. - ETA: 0s - loss: 0.3727 - acc: 0. - ETA: 0s - loss: 0.3726 - acc: 0.956
Epoch 4/25
390/390 [==============================] - 80s 204ms/step - loss: 0.3698 - acc: 0.9578 - val_loss: 0.4351 - val_acc: 0.93970s - loss: 0.3700 - acc: 0.95
Epoch 5/25
390/390 [==============================] - 80s 204ms/step - loss: 0.3693 - acc: 0.9577 - v

In [8]:
# Training for 25 epochs more  - (100-125)
opt_adm = keras.optimizers.Adadelta(lr=0.1*0.1*0.1*0.1)
model.compile(loss='categorical_crossentropy', optimizer=opt_adm, metrics=['accuracy'])
train["part_4"] = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,
                                    verbose=1,validation_data=(x_test,y_test))
model.save("simplenet_trained_model.h5")
print(train["part_4"].history)

Epoch 1/25
390/390 [==============================] - 83s 214ms/step - loss: 0.3579 - acc: 0.9593 - val_loss: 0.4289 - val_acc: 0.9407s: 0.35
Epoch 2/25
390/390 [==============================] - 80s 204ms/step - loss: 0.3547 - acc: 0.9611 - val_loss: 0.4295 - val_acc: 0.9403
Epoch 3/25
390/390 [==============================] - 80s 204ms/step - loss: 0.3573 - acc: 0.9614 - val_loss: 0.4288 - val_acc: 0.9401
Epoch 4/25
390/390 [==============================] - 80s 205ms/step - loss: 0.3567 - acc: 0.9605 - val_loss: 0.4289 - val_acc: 0.9404 5s - loss: 0.3569 -  - ETA: 3s - loss: 0.35
Epoch 5/25
390/390 [==============================] - 80s 206ms/step - loss: 0.3559 - acc: 0.9590 - val_loss: 0.4287 - val_acc: 0.94056 - acc: 0. - ETA: 1s - loss: 0.3556 - acc:
Epoch 6/25
390/390 [==============================] - 80s 205ms/step - loss: 0.3558 - acc: 0.9604 - val_loss: 0.4289 - val_acc: 0.9404: 1:03 - loss: 0.359 - ETA: 1:00  - ETA: 49s - loss: 0.35 - ETA: 48s - loss: 0.35 - ETA:  - ETA: 

In [9]:
print("\n \n Final Logs: ", train)


 
 Final Logs:  {'part_1': <keras.callbacks.History object at 0x000002DBAE21FE80>, 'part_2': <keras.callbacks.History object at 0x000002DBB1226940>, 'part_3': <keras.callbacks.History object at 0x000002DBAE242D68>, 'part_4': <keras.callbacks.History object at 0x000002DBF553F240>}


In [10]:
# Done!